In [1]:
import pandas as pd

data = {
    'User': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'D'],
    'Item': ['Movie1', 'Movie2', 'Movie3', 'Movie1', 'Movie3', 'Movie2', 'Movie4', 'Movie4'],
    'Rating': [5, 3, 4, 4, 5, 2, 5, 4]
}

df = pd.DataFrame(data)
df

,User,Item,Rating
0,A,Movie1,5
1,A,Movie2,3
2,A,Movie3,4
3,B,Movie1,4
4,B,Movie3,5
5,C,Movie2,2
6,C,Movie4,5
7,D,Movie4,4


In [2]:
user_item_matrix = df.pivot_table(
    index='User',
    columns='Item',
    values='Rating'
).fillna(0)

user_item_matrix

Item,Movie1,Movie2,Movie3,Movie4
User,,,,
A,5.0,3.0,4.0,0.0
B,4.0,0.0,5.0,0.0
C,0.0,2.0,0.0,5.0
D,0.0,0.0,0.0,4.0


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

item_similarity = cosine_similarity(user_item_matrix.T)

item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

item_similarity_df

Item,Movie1,Movie2,Movie3,Movie4
Item,,,,
Movie1,1.000000,0.649722,0.975610,0.000000
Movie2,0.649722,1.000000,0.519778,0.433148
Movie3,0.975610,0.519778,1.000000,0.000000
Movie4,0.000000,0.433148,0.000000,1.000000


In [4]:
def get_similar_items(item, similarity_df, n=2):
    return similarity_df[item].sort_values(ascending=False)[1:n+1]

get_similar_items('Movie1', item_similarity_df)

Item
Movie3    0.975610
Movie2    0.649722
Name: Movie1, dtype: float64

In [5]:
def recommend_items_item_based(user, user_item_matrix, similarity_df, n_recommendations=2):
    user_ratings = user_item_matrix.loc[user]
    rated_items = user_ratings[user_ratings > 0]

    scores = pd.Series(dtype=float)

    for item, rating in rated_items.items():
        similar_items = similarity_df[item]
        scores = scores.add(similar_items * rating, fill_value=0)

    scores = scores.drop(rated_items.index, errors='ignore')
    return scores.sort_values(ascending=False).head(n_recommendations)

recommend_items_item_based('A', user_item_matrix, item_similarity_df)

Item
Movie4    1.299444
dtype: float64